In [25]:
# !python3 -m pip install python-dotenv
# !python3 -m pip install NumPy

In [26]:
import os
from hdbcli import dbapi
from dotenv import load_dotenv

In [27]:
# testing out that reading from .env works as expected
load_dotenv()
os.environ.get("TEST")

'mltestbed'

In [28]:
# Connection constants for space PYTHONCODESAMPLES
dbHost="9ee99a70-9d98-481d-ab94-00b0417b66c5.hana.prod-eu10.hanacloud.ondemand.com" #you should have noted them above
dbPort=443
dbUser="SP_PROJECTTEST#PYTHON"
dbPwd=os.environ.get("HANA_PWD_PYTHONCODESAMPLES")  #load pwd from .env file
dbIngestionSchema="SP_PROJECTTEST#PYTHON"
dbConsumptionSchema="SP_PROJECTTEST"
dbExampleTable="V_World_GDP_RAW"
dbExampleView="V_SalesOrder_CV"

In [29]:
#Import your dependencies
import platform
from hdbcli import dbapi

#verify that this is a 64 bit version of Python
print ("Platform architecture: " + platform.architecture()[0])

#Initialize your connection
conn = dbapi.connect(
    address=dbHost,
    port=dbPort,
    user=dbUser,
    password=dbPwd, 
    encrypt=True, # must be set to True when connecting to HANA Cloud
    sslValidateCertificate=False # True HC, False for HANA Express.
)
#If no errors, print connected
print('connected')

Platform architecture: 64bit
connected


In [30]:
cursor = conn.cursor()
sql_command = f'select * from "{dbConsumptionSchema}"."{dbExampleView}"'
print(sql_command)
cursor.execute(sql_command)
rows = cursor.fetchall()
count = 0

for row in rows:
    if count == 10:
        break

    for col in row:
        print("%s" % col, end=" ")
    print(" ")

    count += 1


select * from "SP_PROJECTTEST"."V_SalesOrder_CV"
0500000111 0000000010 RC-1054 None USD I None EA 2018-08-21 0500000111 0000000005 2018-06-12 0000000005 2018-06-16 K4 2018006 None 0100000020 APJ USD C C C 5499 4811.63 687.37 2 71749 62780.45 8968.37  
0500000111 0000000020 MB-1033 None USD I None EA 2018-08-21 0500000111 0000000005 2018-06-12 0000000005 2018-06-16 K4 2018006 None 0100000020 APJ USD C C C 3999 3499.13 499.87 3 71749 62780.45 8968.37  
0500000111 0000000030 BX-1011 None USD I None EA 2018-08-21 0500000111 0000000005 2018-06-12 0000000005 2018-06-16 K4 2018006 None 0100000020 APJ USD C C C 249 217.87 31.12 10 71749 62780.45 8968.37  
0500000111 0000000040 MB-1032 None USD I None EA 2018-08-21 0500000111 0000000005 2018-06-12 0000000005 2018-06-16 K4 2018006 None 0100000020 APJ USD C C C 1299 1136.63 162.37 9 71749 62780.45 8968.37  
0500000111 0000000050 HB-1172 None USD I None EA 2018-08-21 0500000111 0000000005 2018-06-12 0000000005 2018-06-16 K4 2018006 None 0100000020

In [31]:
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext


conn = dataframe.ConnectionContext(address=dbHost,
    port=dbPort,
    user=dbUser, 
    password=dbPwd)

In [51]:
df = conn.sql(sql_command)
data = df.collect()

In [52]:
import pandas
###### Use case (1) Add calculated column 'Deal_Type'########

bins = [0, 500, 1000, 5000, 10000]
labels = ['small deal', 'mid-size deal', 'big deal', 'huge deal']
data['Deal_Type'] = pandas.cut(data['NETAMOUNT'], bins=bins, labels=labels)
data.head(10)


,SALESORDERID,SALESORDERITEM,PRODUCTID,NOTEID,CURRENCY,ITEMATPSTATUS,OPITEMPOS,QUANTITYUNIT,DELIVERYDATE,SALESORDERID_1,...,BILLINGSTATUS,DELIVERYSTATUS,GROSSAMOUNT,NETAMOUNT,TAXAMOUNT,QUANTITY,GROSSAMOUNT_1,NETAMOUNT_1,TAXAMOUNT_1,Deal_Type
0,0500000111,0000000010,RC-1054,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,5499,4811.63,687.37,2,71749,62780.45,8968.37,big deal
1,0500000111,0000000020,MB-1033,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,3999,3499.13,499.87,3,71749,62780.45,8968.37,big deal
2,0500000111,0000000030,BX-1011,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,249,217.87,31.12,10,71749,62780.45,8968.37,small deal
3,0500000111,0000000040,MB-1032,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,1299,1136.63,162.37,9,71749,62780.45,8968.37,big deal
4,0500000111,0000000050,HB-1172,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,799,699.12,99.87,3,71749,62780.45,8968.37,mid-size deal
5,0500000111,0000000060,DB-1081,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,1499,1311.63,187.37,6,71749,62780.45,8968.37,big deal
6,0500000111,0000000070,HB-1171,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,699,611.62,87.37,5,71749,62780.45,8968.37,mid-size deal
7,0500000111,0000000080,RC-1055,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,1999,1749.13,249.87,4,71749,62780.45,8968.37,big deal
8,0500000111,0000000090,MB-1032,None,USD,I,None,EA,2018-08-21,0500000111,...,C,C,1299,1136.63,162.37,9,71749,62780.45,8968.37,big deal
9,0500000360,0000000010,RC-1053,None,USD,I,None,EA,2019-08-08,0500000360,...,C,C,4599,4024.13,574.87,2,56901,49788.27,7112.53,big deal


In [53]:
###### Use case (2a) Segment data in to buckests based on deal type ########

segments = data.groupby("Deal_Type")["Deal_Type"]
segments.describe() 



,count,unique,top,freq
Deal_Type,,,,
small deal,606,1,small deal,606
mid-size deal,446,1,mid-size deal,446
big deal,1338,1,big deal,1338
huge deal,65,1,huge deal,65


In [54]:

###### Use case (2 b) Segment data in to buckests based on deal type ########
# Other ways of doing a same thing, pandas cut, groupby size unstack to segmentation 

sg_bins = pandas.cut(data['NETAMOUNT'], bins)

sg = data.groupby(['Deal_Type', sg_bins])  
sg.size().unstack()


NETAMOUNT,"(0, 500]","(500, 1000]","(1000, 5000]","(5000, 10000]"
Deal_Type,,,,
small deal,606,0,0,0
mid-size deal,0,446,0,0
big deal,0,0,1338,0
huge deal,0,0,0,65


In [55]:
###### Use case (3) Add calculated column 'US_Delivery Date'########
# Data converstion to mm.dd.yyyy format
data['US_DELIVERYDATE'] = pandas.to_datetime(data['DELIVERYDATE'], errors = 'coerce').dt.strftime('%m-%d-%Y')

# pd['US_DELIVERYDATE'] = pd['DELIVERYDATE']

data.head(10)

,SALESORDERID,SALESORDERITEM,PRODUCTID,NOTEID,CURRENCY,ITEMATPSTATUS,OPITEMPOS,QUANTITYUNIT,DELIVERYDATE,SALESORDERID_1,...,DELIVERYSTATUS,GROSSAMOUNT,NETAMOUNT,TAXAMOUNT,QUANTITY,GROSSAMOUNT_1,NETAMOUNT_1,TAXAMOUNT_1,Deal_Type,US_DELIVERYDATE
0,0500000111,0000000010,RC-1054,None,USD,I,None,EA,2018-08-21,0500000111,...,C,5499,4811.63,687.37,2,71749,62780.45,8968.37,big deal,08-21-2018
1,0500000111,0000000020,MB-1033,None,USD,I,None,EA,2018-08-21,0500000111,...,C,3999,3499.13,499.87,3,71749,62780.45,8968.37,big deal,08-21-2018
2,0500000111,0000000030,BX-1011,None,USD,I,None,EA,2018-08-21,0500000111,...,C,249,217.87,31.12,10,71749,62780.45,8968.37,small deal,08-21-2018
3,0500000111,0000000040,MB-1032,None,USD,I,None,EA,2018-08-21,0500000111,...,C,1299,1136.63,162.37,9,71749,62780.45,8968.37,big deal,08-21-2018
4,0500000111,0000000050,HB-1172,None,USD,I,None,EA,2018-08-21,0500000111,...,C,799,699.12,99.87,3,71749,62780.45,8968.37,mid-size deal,08-21-2018
5,0500000111,0000000060,DB-1081,None,USD,I,None,EA,2018-08-21,0500000111,...,C,1499,1311.63,187.37,6,71749,62780.45,8968.37,big deal,08-21-2018
6,0500000111,0000000070,HB-1171,None,USD,I,None,EA,2018-08-21,0500000111,...,C,699,611.62,87.37,5,71749,62780.45,8968.37,mid-size deal,08-21-2018
7,0500000111,0000000080,RC-1055,None,USD,I,None,EA,2018-08-21,0500000111,...,C,1999,1749.13,249.87,4,71749,62780.45,8968.37,big deal,08-21-2018
8,0500000111,0000000090,MB-1032,None,USD,I,None,EA,2018-08-21,0500000111,...,C,1299,1136.63,162.37,9,71749,62780.45,8968.37,big deal,08-21-2018
9,0500000360,0000000010,RC-1053,None,USD,I,None,EA,2019-08-08,0500000360,...,C,4599,4024.13,574.87,2,56901,49788.27,7112.53,big deal,08-08-2019


In [144]:

sql_command_gdp = f'select * from "{dbConsumptionSchema}"."{dbExampleTable}"'
print(sql_command_gdp)
print()
cursor.execute(sql_command_gdp)
rows = cursor.fetchall()
count = 0

for row in rows:
    if count == 10:
        break

    for col in row:
        print("%s" % col, end=" ")
    print(" ")

    count += 1


select * from "SP_PROJECTTEST"."V_World_GDP_RAW"

None None None None None None None None None None None None None None None None None None None None None None None None None None 6283.001443 7567.253642 9274.514156 10767.39622 11638.73371 12850.21571 13657.67064 14970.15234 16675.27849 17140.43337 17375.22531 18713.42539 19742.31674 19833.82675 21023.15751 20913.2995 21377.09519 22050.83093 24104.64618 24975.67326 25833.44562 27665.42647 29011.55925 25739.13725 24452.92836 26044.43593 25609.95572 26515.67808 26942.30798 28421.38649 28451.27374 29326.70806 30220.59452 31650.76054 24487.86356 29342.10086 Aruba ABW  
162.9130345 162.551683 172.00246 199.1892382 179.3877989 198.2303679 209.4146653 211.7070603 224.2397833 249.5891331 260.1210841 278.761746 293.0858157 370.4270504 444.7328661 459.9117509 444.0905654 489.8633813 530.3907202 600.3912121 738.8987409 732.9434186 681.673765 692.0086402 615.4319578 508.7120317 552.5549671 654.337932 697.2411233 722.4517041 817.175973 858.3265971 

In [163]:
df_gdp = conn.sql(sql_command_gdp)
data_gdp = df_gdp.collect()
# data_gdp

In [164]:
###### Use case (4) Unpivot GDP per capital data########
# Data source world bank, data product on market place is not working hence direct .csv upload


data_gdp_LT = data_gdp.melt(id_vars=['Country Code', 'Country Name'], var_name='YEAR', value_name='VALUE')
# data_gdp_LT
data_gdp_LT.head(10)
# pd_gdp_LT.at[1,'Country Code']

,Country Code,Country Name,YEAR,VALUE
0,ABW,Aruba,1960,None
1,AFE,Africa Eastern and Southern,1960,162.9130345
2,AFG,Afghanistan,1960,62.36937451
3,AFW,Africa Western and Central,1960,106.9764749
4,AGO,Angola,1960,None
5,ALB,Albania,1960,None
6,AND,Andorra,1960,None
7,ARB,Arab World,1960,None
8,ARE,United Arab Emirates,1960,None
9,ARG,Argentina,1960,None


In [165]:
import numpy as np
###### Use case (5) Data cleansing  using fillna with bfill, ffill is incorrect in the data context.

# data_gdp_LT.fillna(value=np.NaN , inplace=True)
# data_gdp_LT.fillna(method ='bfill' , inplace=True)

# Correct approach in datascience context is to delete the rows without GDP value

data_gdp_LT.dropna(subset=['VALUE'], inplace=True)

# Round off the GDP values to cieling limit 162.9130345 -> 163

data_gdp_LT['VALUE'] = data_gdp_LT['VALUE'].apply(np.ceil) 
# data_gdp_LT
data_gdp_LT.head(10)

,Country Code,Country Name,YEAR,VALUE
1,AFE,Africa Eastern and Southern,1960,163
2,AFG,Afghanistan,1960,63
3,AFW,Africa Western and Central,1960,107
13,AUS,Australia,1960,1811
14,AUT,Austria,1960,936
16,BDI,Burundi,1960,72
17,BEL,Belgium,1960,1274
18,BEN,Benin,1960,91
19,BFA,Burkina Faso,1960,70
20,BGD,Bangladesh,1960,85


In [166]:
###### Use case (6) Data Manipulation, Change Canada GDP value for 2021 to 9999.

df_before = data_gdp_LT.loc[(data_gdp_LT['Country Code'] == 'CAN') & (data_gdp_LT['YEAR'] == '2021')]

# Get access to the row Canada GDP 2021
print(df_before)

# Get access to the row Canada GDP 2021 and change VALUE only to 9999
data_gdp_LT.loc[(data_gdp_LT['Country Code'] == 'CAN') & (data_gdp_LT['YEAR'] == '2021'), 'VALUE'] = 9999

# Get access to the row Canada GDP 2021
df_after = data_gdp_LT.loc[(data_gdp_LT['Country Code'] == 'CAN') & (data_gdp_LT['YEAR'] == '2021')]
print(df_after)


      Country Code Country Name  YEAR  VALUE
16261          CAN       Canada  2021  51988
      Country Code Country Name  YEAR VALUE
16261          CAN       Canada  2021  9999
